## 데이터

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import sqlite3
import re


In [5]:
df_2015 = pd.read_csv('/content/drive/MyDrive/1_public/markets_data/markets_2015.csv', encoding='cp949')

In [6]:
df_2015.head()

,시장명,시장유형,소재지도로명주소,시장개설주기,소재지지번주소,점포수,사용가능상품권,공중화장실 보유여부,주차장 보유여부,개설년도,데이터기준일자
0,중앙시장,상설,강원도 강릉시 금성로 21,상설,강원 강릉시 성남동 50번지,229,온누리상품권,Y,Y,1980.0,2015-12-31
1,강릉동부시장,상설,강원도 강릉시 옥천로 48,상설,강원 강릉시 옥천동 223번지,91,NaN,Y,N,2014.0,2015-12-31
2,강릉서부시장,상설,강원도 강릉시 임영로 155번길 6,상설,강원 강릉시 용강동 29번지,109,온누리상품권,Y,Y,1977.0,2015-12-31
3,주문진종합시장,상설+정기,강원도 강릉시 주문진읍 구 시장길,5일,강원 강릉시 주문진읍 주문리 321-52번지,77,온누리상품권,Y,Y,2006.0,2015-12-31
4,주문진수산시장,상설,강원도 강릉시 주문진읍 시장1길 4-1,상설,강원 강릉시 주문진읍 주문리 312-91번지,42,온누리상품권,Y,Y,2007.0,2015-12-31


In [7]:
df_2015.columns

Index(['시장명', '시장유형', '소재지도로명주소', '시장개설주기', '소재지지번주소', '점포수', '사용가능상품권',
       '공중화장실 보유여부', '주차장 보유여부', '개설년도', '데이터기준일자'],
      dtype='object')

In [8]:
df_2024 = pd.read_csv('/content/drive/MyDrive/1_public/markets_data/markets_2024.csv', encoding='euc-kr') # utf-8

In [9]:
df_2024.columns

Index(['시장코드', '시장명', '시장 유형', '지번주소', '도로명주소', '시도', '시군구', '아케이드 보유 여부',
       '엘리베이터_에스컬레이터_보유여부', '고객지원센터 보유 여부', '스프링쿨러 보유 여부', '화재감지기 보유여부',
       '유아놀이방_보유여부', '종합콜센터_보유여부', '고객휴게실_보유여부', '수유센터_보유여부', '물품보관함_보유여부',
       '자전거보관함_보유여부', '체육시설_보유여부', '간이 도서관_보유여부', '쇼핑카트_보유여부', '외국인 안내센터_보유여부',
       '고객동선통로_보유여부', '방송센터_보유여부', '문화교실_보유여부', '공동물류창고_보유여부',
       '시장전용 고객주차장_보유여부', '교육장_보유여부', '회의실_보유여부', '자동심장충격기_보유여부'],
      dtype='object')

In [10]:
df_2024.head()

,시장코드,시장명,시장 유형,지번주소,도로명주소,시도,시군구,아케이드 보유 여부,엘리베이터_에스컬레이터_보유여부,고객지원센터 보유 여부,...,쇼핑카트_보유여부,외국인 안내센터_보유여부,고객동선통로_보유여부,방송센터_보유여부,문화교실_보유여부,공동물류창고_보유여부,시장전용 고객주차장_보유여부,교육장_보유여부,회의실_보유여부,자동심장충격기_보유여부
0,b31370001,설악눈메골시장,전통시장,경기도 가평군 설악면 신천리 413-14,경기도 가평군 설악면 신천중앙로 104-1,경기도,가평군,N,N,N,...,N,N,N,N,N,N,Y,Y,Y,N
1,a31370001,가평잣고을시장,전통시장,경기도 가평군 읍내리 405,경기도 가평군 가평읍 장터2길12 204호,경기도,가평군,Y,N,Y,...,N,N,Y,Y,N,Y,Y,Y,Y,N
2,a31370007,청평여울시장,전통시장,경기도 가평군 청평면 청평리 81-2,경기도 가평군 청평면 시장중앙로19,경기도,가평군,Y,Y,N,...,N,N,N,N,N,N,Y,N,Y,N
3,a31100001,원당시장,전통시장,경기도 고양시 덕양구 성사동 697-14,경기도 고양시 덕양구 호국로 790번길 17,경기도,고양시,Y,N,Y,...,N,N,Y,Y,N,N,Y,Y,Y,Y
4,a31101001,능곡시장,전통시장,경기도 고양시 덕양구 토당동 347,경기도 고양시 덕양구 능곡로 12의 9,경기도,고양시,N,N,Y,...,Y,N,Y,Y,N,Y,Y,Y,Y,Y


In [11]:
df_st_2024 = pd.read_csv('/content/drive/MyDrive/1_public/markets_data/markets_st_2024.csv', encoding='euc-kr') # utf-8

In [12]:
df_st_2024.columns

Index(['시장명', '시장유형', '소재지도로명주소', '소재지지번주소', '시장개설주기', '위도', '경도', '점포수',
       '취급품목', '사용가능상품권', '홈페이지주소', '공중화장실보유여부', '주차장보유여부', '개설연도', '전화번호',
       '데이터기준일자', '제공기관코드', '제공기관명'],
      dtype='object')

In [ ]:
df_st_2024.head()

,시장명,시장유형,소재지도로명주소,소재지지번주소,시장개설주기,위도,경도,점포수,취급품목,사용가능상품권,홈페이지주소,공중화장실보유여부,주차장보유여부,개설연도,전화번호,데이터기준일자,제공기관코드,제공기관명
0,서문시장,상설장,충청북도 청주시 상당구 남사로 89번길 61,충청북도 청주시 상당구 서문동 166-43,매일,36.635116,127.485366,106.0,축산물,NaN,NaN,Y,Y,1964.0,NaN,2024-11-04,B553077,소상공인시장진흥공단
1,육거리종합시장,상설장+5일장,충청북도 청주시 상당구 청남로2197번길 42,충청북도 청주시 상당구 석교동 60-4,2일+7일,36.627941,127.488184,385.0,농산물+축산물+수산물+가공식품+의류+신발+가정용품+음식점,NaN,NaN,Y,Y,1970.0,043-222-6696,2024-11-04,B553077,소상공인시장진흥공단
2,사직시장,상설장,충청북도 청주시 서원구 사직대로 265번길 7,충청북도 청주시 서원구 사직동 267-12,매일,36.636142,127.476875,55.0,농산물+축산물,NaN,NaN,Y,Y,1981.0,NaN,2024-11-04,B553077,소상공인시장진흥공단
3,청주두꺼비시장(구수곡시장),상설장,충청북도 청주시 서원구 수곡로 58번길 7-2,충청북도 청주시 서원구 수곡동 862,매일,36.617441,127.472993,70.0,농산물+축산물+수산물+가공식품+의류+신발+가정용품+음식점,NaN,NaN,Y,N,1997.0,NaN,2024-11-04,B553077,소상공인시장진흥공단
4,원마루시장,상설장,충청북도 청주시 서원구 원마루로8번길 21,충청북도 청주시 서원구 분평동 1335,매일,36.609037,127.496414,120.0,농산물+축산물+가공식품+의류+신발+가정용품+음식점,NaN,NaN,Y,N,1998.0,070-7198-4129,2024-11-04,B553077,소상공인시장진흥공단


---

## 시장명 서로 일치하는지 확인

In [25]:
def clean_market_name(name):
    # 1. 괄호 안 내용 제거 (예: "상남시장(상설+5일장)" → "상남시장")
    name = re.sub(r'\([^)]*\)', '', name)
    # 2. 느낌표나 특수기호 제거 (예: "시장!" → "시장")
    name = re.sub(r'[^\w가-힣]', '', name)  # 한글/영문/숫자 외 제거
    # 3. 공백 및 하이픈 제거 (이미 2에서 대부분 처리되지만 추가로 안전 조치)
    name = name.strip().replace(' ', '').replace('-', '')
    return name

# 우선 각 DataFrame에서 '시장명' 열 추출
market_names_st_2024 = set(df_st_2024['시장명'].map(clean_market_name))
market_names_2024 = set(df_2024['시장명'].map(clean_market_name))
market_names_2015 = set(df_2015['시장명'].map(clean_market_name))

# 1. 각 쌍끼리 공통된 시장명
common_st_and_2024 = market_names_st_2024 & market_names_2024
common_st_and_2015 = market_names_st_2024 & market_names_2015
common_2024_and_2015 = market_names_2024 & market_names_2015

# 2. 세 집합 전체 공통 시장명
common_all = market_names_st_2024 & market_names_2024 & market_names_2015

# 3. 집합 간 차이 확인 (어떤 시장이 누락됐는지 등)
only_in_st_2024 = market_names_st_2024 - (market_names_2024 | market_names_2015)
only_in_2024 = market_names_2024 - (market_names_st_2024 | market_names_2015)
only_in_2015 = market_names_2015 - (market_names_st_2024 | market_names_2024)

# 결과 출력 예시
print(f"세 파일 모두에 공통으로 존재하는 시장 수: {len(common_all)}")
print(f"st_2024에만 존재하는 시장 수: {len(only_in_st_2024)}")
print(f"2024에만 존재하는 시장 수: {len(only_in_2024)}")
print(f"2015에만 존재하는 시장 수: {len(only_in_2015)}")

세 파일 모두에 공통으로 존재하는 시장 수: 1065
st_2024에만 존재하는 시장 수: 0
2024에만 존재하는 시장 수: 0
2015에만 존재하는 시장 수: 292


In [26]:
print("st_2024에만 있는 시장:", only_in_st_2024)
print("2024에만 있는 시장:", only_in_2024)
print("2015에만 있는 시장:", only_in_2015)

st_2024에만 있는 시장: set()
2024에만 있는 시장: set()
2015에만 있는 시장: {'부일종합시장', '역촌중앙시장', '연무시장', '고강시장', '현대타운상가', '고현종합시장', '개봉프라자', '르네시떼', '회천5일시장', '거성시장', '노화읍5일시장', '선산종합시장', '내토재래시장', '서호시장', '천호시장', '도림시장', '영동교골목시장', '원종시장', '목4동시장', '대림골목시장', '부평자유시장', '강북종합전통시장', '갈산정기시장', '중곡제일골목시장', '성원3차상가', '진천전통시장', '금호시장', '동진시장', '진상5일시장', '감곡전통시장', '광명시장', '서교시장', '운산공설시장', '당진시장', '합정시장', '인천용현시장', '고현공설시장', '창평전통시장', '하대원시장', '마천전통시장', '임회면십일시장', '구천동공구상가', '연무안심시장', '송북시장', '대화전통시장', '사당시장', '지도시장', '화곡중앙골목시장', '용문종합시장', '다인시장', '익산신동시장', '영동중앙시장', '송강시장', '전주동부시장', '오산오색시장', '그린피아상가', '노룬산골목시장', '중마상설시장', '천안공설시장', '후평일단지시장', '상하시장', '논현종합시장', '서정시장', '이현시장', '대구대동시장', '고덕전통시장', '부산진남문시장', '개봉중앙시장', '완도중앙시장', '새구포종합상가시장', '수정재래시장', '연수종합상가', '번동북부시장', '화천재래시장', '예당5일시장', '야탑에이스코아시장', '진주장대시장', '태안서부재래시장', '농산물종합시장', '오이도종합어시장', '성정5단지시장', '정산공설시장', '옥포시장', '봉곡시장', '새당감시장', '칠성로상가', '신성시장', '신림종합시장', '석관황금시장', '안지랑시장', '북부산시장', '용문전통시장', '삼방전통시장', '동소문스카이상가', '동북시장', '병방시장', '남목시티시장', '울진시장'

In [28]:
for market_2015 in only_in_2015:
    for market in list(df_st_2024['시장명']) + list(df_2024['시장명']):
        if clean_market_name(market_2015) in clean_market_name(market):
            print(f"비슷한 시장명: {market_2015} → {market}")

비슷한 시장명: 연무시장 → 반딧불이연무시장
비슷한 시장명: 연무시장 → 반딧불이연무시장
비슷한 시장명: 르네시떼 → 르네시떼시장
비슷한 시장명: 르네시떼 → 르네시떼시장
비슷한 시장명: 진천전통시장 → 운수대통 생거진천전통시장
비슷한 시장명: 진천전통시장 → 운수대통! 생거진천전통시장
비슷한 시장명: 구천동공구상가 → 구천동공구상가시장
비슷한 시장명: 구천동공구상가 → 구천동공구상가시장
비슷한 시장명: 그린피아상가 → 덕산그린피아상가
비슷한 시장명: 그린피아상가 → 덕산그린피아상가
비슷한 시장명: 농산물종합시장 → 충주농산물종합시장(중앙청과시장)
비슷한 시장명: 농산물종합시장 → 충주농산물종합시장(중앙청과시장)
비슷한 시장명: 건어물시장 → 양동건어물시장
비슷한 시장명: 건어물시장 → 주문진건어물시장
비슷한 시장명: 건어물시장 → 양동건어물시장
비슷한 시장명: 건어물시장 → 주문진건어물시장
비슷한 시장명: 거북시장 → 신거북시장
비슷한 시장명: 거북시장 → 장안문거북시장
비슷한 시장명: 거북시장 → 장안문거북시장
비슷한 시장명: 거북시장 → 신거북시장
비슷한 시장명: 펭귄시장 → 조원동펭귄시장
비슷한 시장명: 펭귄시장 → 조원동펭귄시장
비슷한 시장명: 춘양시장 → 억지춘양시장
비슷한 시장명: 춘양시장 → 억지춘양시장
비슷한 시장명: 동시장 → 항동시장
비슷한 시장명: 동시장 → 인동시장
비슷한 시장명: 동시장 → 광주양동시장
비슷한 시장명: 동시장 → 서동시장
비슷한 시장명: 동시장 → 교동시장
비슷한 시장명: 동시장 → 감천2동시장
비슷한 시장명: 동시장 → 경동시장
비슷한 시장명: 동시장 → 부천상동시장
비슷한 시장명: 동시장 → 목2동시장
비슷한 시장명: 동시장 → 화곡본동시장
비슷한 시장명: 동시장 → 서동시장
비슷한 시장명: 동시장 → 지동시장
비슷한 시장명: 동시장 → 대동시장
비슷한 시장명: 동시장 → 진동시장
비슷한 시장명: 동시장 → 육동시장
비슷한 시장명: 동시장 → 교동시장
비슷한 시장명: 동시장 → 영동시장
비슷한 시장명: 동시장 → 인

## 시장명 교체하기

### ✅ 시장명 교체 목록
| 기존 시장명  | 교체할 시장명           |
| ------- | ----------------- |
| 연무시장    | 반딧불이연무시장          |
| 르네시떼    | 르네시떼시장            |
| 진천전통시장  | 운수대통 생거진천전통시장     |
| 구천동공구상가 | 구천동공구상가시장         |
| 그린피아상가  | 덕산그린피아상가          |
| 농산물종합시장 | 충주농산물종합시장(중앙청과시장) |
| 펭귄시장    | 조원동펭귄시장           |
| 춘양시장    | 억지춘양시장            |
| 사북시장    | 정선사북시장            |
| 아랫장     | 순천시아랫장(순천아랫장)     |
| 큰동해시장   | 포항큰동해시장           |
| 유신종합상가  | 양곡유신종합상가          |
| 성원그랜드쇼핑 | 성원그랜드쇼핑상가         |
| 선어시장    | 중앙선어시장            |
| 토월복합상가  | 토월복합상가시장          |
| 호계종합시장  | 안양호계종합시장          |
| 온천시장    | 온양온천시장(온양전통시장)    |

### ❌삭제 대상 시장명
건어물시장

거북시장

동시장

문화시장

골목시장

남문시장

복대시장



In [29]:
# 교체할 시장명 매핑 (기존 이름 → 새 이름)
market_name_replace_map = {
    "연무시장": "반딧불이연무시장",
    "르네시떼": "르네시떼시장",
    "진천전통시장": "운수대통 생거진천전통시장",
    "구천동공구상가": "구천동공구상가시장",
    "그린피아상가": "덕산그린피아상가",
    "농산물종합시장": "충주농산물종합시장(중앙청과시장)",
    "펭귄시장": "조원동펭귄시장",
    "춘양시장": "억지춘양시장",
    "사북시장": "정선사북시장",
    "아랫장": "순천시아랫장(순천아랫장)",
    "큰동해시장": "포항큰동해시장",
    "유신종합상가": "양곡유신종합상가",
    "성원그랜드쇼핑": "성원그랜드쇼핑상가",
    "선어시장": "중앙선어시장",
    "토월복합상가": "토월복합상가시장",
    "호계종합시장": "안양호계종합시장",
    "온천시장": "온양온천시장(온양전통시장)"
}

# 삭제할 시장명 목록
market_names_to_drop = {
    "건어물시장", "거북시장", "동시장", "문화시장",
    "골목시장", "남문시장", "복대시장"
}

df_2015_rn = df_2015.copy()

# 시장명 교체
df_2015_rn['시장명'] = df_2015_rn['시장명'].replace(market_name_replace_map)

# 삭제 대상 행 필터링
df_2015_rn = df_2015_rn[~df_2015_rn['시장명'].isin(market_names_to_drop)].reset_index(drop=True)

print("시장명 변경 및 삭제 후 DataFrame 크기:", df_2015_rn.shape)
print("시장명 샘플 확인:")
print(df_2015_rn['시장명'].head(10))

시장명 변경 및 삭제 후 DataFrame 크기: (1431, 11)
시장명 샘플 확인:
0        중앙시장
1      강릉동부시장
2      강릉서부시장
3     주문진종합시장
4     주문진수산시장
5    주문진건어물시장
6      강릉성남시장
7        간성시장
8        거진시장
9      북평민속시장
Name: 시장명, dtype: object


## 각 데이터 프레임에서 정제해서 시장 이름 통일

In [30]:
# 각 DataFrame에서 '시장명' 열 추출해서 시장 이름 통일
df_st_2024['시장명'] = df_st_2024['시장명'].map(clean_market_name)
df_2024['시장명'] = df_2024['시장명'].map(clean_market_name)
df_2015_rn['시장명'] = df_2015_rn['시장명'].map(clean_market_name)

## 시장명 기준으로 병합

In [ ]:
# 우선 각 DataFrame에 출처 구분을 위한 접두어 추가 (선택적이지만 권장)
df_st_2024 = df_st_2024.add_prefix('st_')
df_2024 = df_2024.add_prefix('d24_')
df_2015_rn = df_2015_rn.add_prefix('d15_')

# 공통 키 컬럼 이름을 동일하게 정리 ('시장명')
df_st_2024 = df_st_2024.rename(columns={'st_시장명': '시장명'})
df_2024 = df_2024.rename(columns={'d24_시장명': '시장명'})
df_2015_rn = df_2015_rn.rename(columns={'d15_시장명': '시장명'})

### df_2024와 df_2015 비교

| 컬럼명          | 설명             | df\_2024에 있는지?                  | 추천 사유                      |
| ------------ | -------------- | ------------------------------- | -------------------------- |
| `시장유형`       | 시장의 분류         | ✔ `시장 유형`으로 존재 (컬럼명만 다름)        | ❌ 이미 있음                    |
| `소재지도로명주소`   | 주소             | ✔ `도로명주소`로 존재                   | ❌ 중복                       |
| `소재지지번주소`    | 주소 (지번)        | ✔ `지번주소`로 존재                    | ❌ 중복                       |
| `시장개설주기`     | 정기시장 여부        | ❌ 없음                            | ✅ **정기시장/상설시장 여부**는 유의미    |
| `점포수`        | 점포 개수          | ❌ 없음                            | ✅ **시장 규모 판단 지표**로 유의미     |
| `사용가능상품권`    | 온누리상품권 등 가능 여부 | ❌ 없음                            | ✅ **결제 수단 다양성 정보**로 유용     |
| `공중화장실 보유여부` | 편의시설 정보        | ❌ 없음                            | ✅ **화장실 여부는 핵심 편의시설** 중 하나 |
| `주차장 보유여부`   | 주차장 유무         | ✔ 비슷한 항목 존재 (`시장전용 고객주차장_보유여부`) | ⚠️ **유사**, 비교용으로는 OK       |
| `개설년도`       | 시장 연도          | ❌ 없음                            | ✅ **시장 연령** 분석에 유용         |
| `데이터기준일자`    | 수집 시점          | ❌ 없음                            | ⚠️ 덜 중요, 보조 정보용            |


In [ ]:
# 필요한 컬럼만 선택하여 병합
merged1 = pd.merge(
    df_2024,
    df_2015[['시장명', '공중화장실 보유여부', '주차장 보유여부', '개설년도']],
    on='시장명',
    how='left'
)

print(merged1.columns)

In [ ]:
# 먼저 두 개 병합
merged_df = pd.merge(df_st_2024, df_2024, on='시장명', how='outer')

# 그 결과에 2015 데이터 병합
merged_df = pd.merge(merged_df, df_2015_rn, on='시장명', how='outer')

# 결과 확인
print("병합된 데이터프레임 크기:", merged_df.shape)
merged_df.head()

In [33]:
merged_df.columns

Index(['시장명', 'st_시장유형', 'st_소재지도로명주소', 'st_소재지지번주소', 'st_시장개설주기', 'st_위도',
       'st_경도', 'st_점포수', 'st_취급품목', 'st_사용가능상품권', 'st_홈페이지주소', 'st_공중화장실보유여부',
       'st_주차장보유여부', 'st_개설연도', 'st_전화번호', 'st_데이터기준일자', 'st_제공기관코드',
       'st_제공기관명', 'd24_시장코드', 'd24_시장 유형', 'd24_지번주소', 'd24_도로명주소', 'd24_시도',
       'd24_시군구', 'd24_아케이드 보유 여부', 'd24_엘리베이터_에스컬레이터_보유여부',
       'd24_고객지원센터 보유 여부', 'd24_스프링쿨러 보유 여부', 'd24_화재감지기 보유여부',
       'd24_유아놀이방_보유여부', 'd24_종합콜센터_보유여부', 'd24_고객휴게실_보유여부', 'd24_수유센터_보유여부',
       'd24_물품보관함_보유여부', 'd24_자전거보관함_보유여부', 'd24_체육시설_보유여부', 'd24_간이 도서관_보유여부',
       'd24_쇼핑카트_보유여부', 'd24_외국인 안내센터_보유여부', 'd24_고객동선통로_보유여부',
       'd24_방송센터_보유여부', 'd24_문화교실_보유여부', 'd24_공동물류창고_보유여부',
       'd24_시장전용 고객주차장_보유여부', 'd24_교육장_보유여부', 'd24_회의실_보유여부',
       'd24_자동심장충격기_보유여부', 'd15_시장유형', 'd15_소재지도로명주소', 'd15_시장개설주기',
       'd15_소재지지번주소', 'd15_점포수', 'd15_사용가능상품권', 'd15_공중화장실 보유여부',
       'd15_주차장 보유여부', 'd15_개설년도', 'd15_데이터기준일자'],
      dtype='object')

## 중복된 의미의 컬럼 후보 (같은 정보, 다른 이름)

| 공통 의미      | `st_`         | `d24_`     | `d15_`            |
| ---------- | ------------- | ---------- | ----------------- |
| 시장유형       | st\_시장유형      | d24\_시장 유형 | d15\_시장유형         |
| 도로명주소      | st\_소재지도로명주소  | d24\_도로명주소 | d15\_소재지도로명주소? 없음 |
| 지번주소       | st\_소재지지번주소   | d24\_지번주소  | d15\_소재지지번주소      |
| 시장개설주기     | st\_시장개설주기    | -          | d15\_시장개설주기       |
| 점포수        | st\_점포수       | -          | d15\_점포수          |
| 사용가능상품권    | st\_사용가능상품권   | -          | d15\_사용가능상품권      |
| 공중화장실 보유여부 | st\_공중화장실보유여부 | -          | d15\_공중화장실 보유여부   |
| 주차장 보유여부   | st\_주차장보유여부   | -          | d15\_주차장 보유여부     |
| 개설년도       | st\_개설연도      | -          | d15\_개설년도         |
| 데이터기준일자    | st\_데이터기준일자   | -          | d15\_데이터기준일자      |

✅ 정리
데이터 시점 차이, 정보 최신성, 분석 목적 을 고려해서 유지/삭제/통합 여부를 결정

In [36]:
# 중복 후보 컬럼 쌍 정의
duplicate_candidates = [
    ('st_시장유형', 'd15_시장유형'),
    ('st_소재지도로명주소', 'd15_소재지도로명주소'),
    ('st_소재지지번주소', 'd15_소재지지번주소'),
    ('st_시장개설주기', 'd15_시장개설주기'),
    ('st_점포수', 'd15_점포수'),
    ('st_사용가능상품권', 'd15_사용가능상품권'),
    ('st_공중화장실보유여부', 'd15_공중화장실 보유여부'),
    ('st_주차장보유여부', 'd15_주차장 보유여부'),
    ('st_개설연도', 'd15_개설년도'),
    ('st_데이터기준일자', 'd15_데이터기준일자'),
]

# 같으면 제거 대상 목록 생성
print("✅ df_st_2024 vs df_2015 컬럼 값 비교 결과\n")
to_drop = []
for col1, col2 in duplicate_candidates:
    if col1 in merged_df.columns and col2 in merged_df.columns:
        if merged_df[col1].equals(merged_df[col2]):
            print(f"✅ 중복 컬럼 (삭제 가능): {col2} (→ {col1}과 동일)")
            to_drop.append(col2)
        else:
            print(f"⚠️ 다른 값 존재: {col1} ≠ {col2}")

✅ df_st_2024 vs df_2015 컬럼 값 비교 결과

⚠️ 다른 값 존재: st_시장유형 ≠ d15_시장유형
⚠️ 다른 값 존재: st_소재지도로명주소 ≠ d15_소재지도로명주소
⚠️ 다른 값 존재: st_소재지지번주소 ≠ d15_소재지지번주소
⚠️ 다른 값 존재: st_시장개설주기 ≠ d15_시장개설주기
⚠️ 다른 값 존재: st_점포수 ≠ d15_점포수
⚠️ 다른 값 존재: st_사용가능상품권 ≠ d15_사용가능상품권
⚠️ 다른 값 존재: st_공중화장실보유여부 ≠ d15_공중화장실 보유여부
⚠️ 다른 값 존재: st_주차장보유여부 ≠ d15_주차장 보유여부
⚠️ 다른 값 존재: st_개설연도 ≠ d15_개설년도
⚠️ 다른 값 존재: st_데이터기준일자 ≠ d15_데이터기준일자


| 컬럼                                                  | 차이 원인 추정                                                      | 처리 전략                                                                 |
| --------------------------------------------------- | ------------------------------------------------------------- | --------------------------------------------------------------------- |
| **시장유형**<br>(st\_시장유형 ≠ d15\_시장유형)                  | - 표기 방식 차이 (예: `상설` vs `상설+5일장`)<br>- 시장 운영 방식 변화 가능성         | `상설`, `5일장`, `복합` 등으로 분류 기준 통일 후 비교 (표준화 필요)<br>→ 또는 `시장유형_비교` 파생열 생성 |
| **소재지도로명주소**<br>(st\_소재지도로명주소 ≠ d15\_소재지도로명주소)      | - 주소 체계 개편 (도로명 주소 변화)<br>- 표기 차이 (지번 병합 등)                   | 도로명주소는 최신 기준(`st_`)을 신뢰하고 사용<br>→ `d15_`는 비교용/보조 정보로 유지               |
| **소재지지번주소**<br>(st\_소재지지번주소 ≠ d15\_소재지지번주소)         | - 행정동 통합/분리, 지번 체계 변화 가능                                      | `st_` 기준 유지, `d15_`는 병렬 비교용                                           |
| **시장개설주기**<br>(st\_시장개설주기 ≠ d15\_시장개설주기)            | - 실제 시장 형태 변경 (5일장 추가/삭제 등)<br>- 오기록 가능성                      | `시장유형`과 함께 묶어서 비교 분석 → 복합시장 여부 추론 가능                                  |
| **점포수**<br>(st\_점포수 ≠ d15\_점포수)                     | - 실제 점포 증감 (건물 리모델링, 상점 철수 등)<br>- 집계 기준 변화 가능성               | `점포수_증감 = st_점포수 - d15_점포수`로 파생 열 생성 후 분석                             |
| **사용가능상품권**<br>(st\_사용가능상품권 ≠ d15\_사용가능상품권)         | - 상품권 정책 변화 (온누리 → 지역화폐 등)<br>- 복수 표기 차이                      | `사용가능상품권`을 리스트화 후 `포함 여부` 기준 비교                                       |
| **공중화장실 보유여부**<br>(st\_공중화장실보유여부 ≠ d15\_공중화장실 보유여부) | - 실제 설치/철거<br>- 표기 방식(`Y/Yes`, `유/있음` 등) 차이                   | 값 통일 후 비교 (`bool` 또는 `'유/무'`로 정제), 변화 여부만 분석                          |
| **주차장 보유여부**<br>(st\_주차장보유여부 ≠ d15\_주차장 보유여부)       | - 위와 동일                                                       | 위와 동일 방식으로 정제 후 비교                                                    |
| **개설연도**<br>(st\_개설연도 ≠ d15\_개설년도)                  | - 기록 방식 불일치 (`1998` vs `"1998년"`, `NaN`)<br>- 실제 등록 연도 오류 가능성 | 수치형으로 정제 (`int`) 후 비교<br>→ 차이 발생 시장만 수작업 검토                           |
| **데이터기준일자**<br>(st\_데이터기준일자 ≠ d15\_데이터기준일자)         | - 단순히 수집 시점이 다름 (2015 vs 2024)                                | 최신 기준(`st_`)을 기준으로 사용<br>`d15_`는 보조 지표로만 활용                           |


In [46]:
# 예시: 시장유형 비교
diff = merged_df[merged_df['st_사용가능상품권'] != merged_df['d15_사용가능상품권']]
print("시장유형 다른 시장 수:", len(diff))
display(diff[['시장명', 'st_사용가능상품권', 'd15_사용가능상품권']])

print(merged_df['st_사용가능상품권'].unique())
print(merged_df['d15_사용가능상품권'].unique())

시장유형 다른 시장 수: 5479


,시장명,st_사용가능상품권,d15_사용가능상품권
0,1913송정역시장,NaN,NaN
1,Team204,NaN,NaN
2,가경터미널시장,NaN,온누리상품권
3,가락타운상가시장,NaN,온누리상품권
4,가리봉시장,NaN,온누리상품권
...,...,...,...
5474,후포시장,NaN,온누리상품권
5475,흥덕시장,NaN,온누리상품권
5476,흥덕전통시장,NaN,NaN
5477,흥부시장,NaN,온누리상품권


[nan]
[nan '온누리상품권']


In [38]:
# 중복 후보 컬럼 쌍 정의
duplicate_candidates_st_24 = [
    ('st_시장유형', 'd24_시장 유형'),
    ('st_소재지도로명주소', 'd24_도로명주소'),
    ('st_소재지지번주소', 'd24_지번주소'),
    ('st_소재지지번주소', 'd24_지번주소'),
    ('st_소재지도로명주소', 'd24_도로명주소'),
]

# 같으면 제거 대상 목록 생성
print("✅ df_st_2024 vs df_2024 컬럼 값 비교 결과\n")

to_drop = []
for col1, col2 in duplicate_candidates_st_24:
    if col1 in merged_df.columns and col2 in merged_df.columns:
        if merged_df[col1].equals(merged_df[col2]):
            print(f"✅ 중복 컬럼 (삭제 가능): {col2} (→ {col1}과 동일)")
            to_drop.append(col2)
        else:
            print(f"⚠️ 다른 값 존재: {col1} ≠ {col2}")

✅ df_st_2024 vs df_2024 컬럼 값 비교 결과

⚠️ 다른 값 존재: st_시장유형 ≠ d24_시장 유형
⚠️ 다른 값 존재: st_소재지도로명주소 ≠ d24_도로명주소
⚠️ 다른 값 존재: st_소재지지번주소 ≠ d24_지번주소
⚠️ 다른 값 존재: st_소재지지번주소 ≠ d24_지번주소
⚠️ 다른 값 존재: st_소재지도로명주소 ≠ d24_도로명주소


In [44]:
# 예시: 시장유형 비교
diff = merged_df[merged_df['st_시장유형'] != merged_df['d24_시장 유형']]
print("시장유형 다른 시장 수:", len(diff))
display(diff[['시장명', 'st_시장유형', 'd24_시장 유형']])

print(merged_df['d24_시장 유형'].unique())
print(merged_df['st_시장유형'].unique())

시장유형 다른 시장 수: 5479


,시장명,st_시장유형,d24_시장 유형
0,1913송정역시장,상설장,전통시장
1,Team204,상설장,전통시장
2,가경터미널시장,상설장,전통시장
3,가락타운상가시장,상설장,전통시장
4,가리봉시장,상설장,전통시장
...,...,...,...
5474,후포시장,상설장+5일장,전통시장
5475,흥덕시장,NaN,NaN
5476,흥덕전통시장,5일장,전통시장
5477,흥부시장,상설장+5일장,전통시장


['전통시장' nan]
['상설장' '상설장+5일장' '5일장' nan]


In [39]:
# 예시: 점포수 비교
diff = merged_df[merged_df['st_소재지도로명주소'] != merged_df['d24_도로명주소']]
print("도로명 다른 시장 수:", len(diff))
display(diff[['시장명', 'st_소재지도로명주소', 'd24_도로명주소']].head())

점포수 다른 시장 수: 4054


,시장명,st_소재지도로명주소,d24_도로명주소
12,가음시장대상가,NaN,NaN
15,가평잣고을시장,경기도 가평군 가평읍 장터2길12 204호,경기도 가평군 가평읍 장터2길12 204호
16,가평잣고을전통시장,NaN,NaN
18,간성시장,NaN,NaN
19,간성전통시장,강원특별자치도 고성군 간성시장3길 7-2,강원도 고성군 간성시장3길 7-2


# 프로세스

In [ ]:
# 크롤링 함수 작성
def get_review_data(market_name):
    url = f"https://map.naver.com/v5/search/{market_name}"
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(response.text, 'html.parser')
    rating, reviews = None, None
    try:
        rating = soup.select_one('.rating_class').text
        reviews = soup.select_one('.review_count_class').text
    except:
        pass
    return rating, reviews

In [ ]:
# 리뷰 데이터 병합
market_df[['평점', '리뷰수']] = market_df['시장명'].apply(lambda x: pd.Series(get_review_data(x)))

In [ ]:
# 저장
conn = sqlite3.connect('market_review.db')
market_df.to_sql('market_info', conn, if_exists='replace', index=False)
conn.close()

---

### 인코딩 감지

In [ ]:
!pip install chardet

In [ ]:
import chardet

with open('/content/drive/MyDrive/1_public/markets_data/markets_st_2024.csv', 'rb') as f:
    rawdata = f.read(10000)
    result = chardet.detect(rawdata)
    print(result)

---